    Primary goal of few-shot learning: learn a similarity function that maps the similarities between classes in the support and query sets. 
    Example selector can be designed to:
        1. choose a set of relevant examples that are representative of the desired output.
        2. select a subset of examples that will be most informative for the language model. 
    LengthBasedExampleSelector-> useful when concerned about the length of the context window
                                 selects fewer examples for longer queries and more examples for shorter queries.

In [1]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

In [2]:
# Define examples and the example_prompt
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]
example_template=""" Word: {word}, Antonym: {antonym} """

example_prompt=PromptTemplate(input_variables=["word", "antonym"], template=example_template)

In [4]:
# Create an instance of LengthBasedExampleSelector
example_selector=LengthBasedExampleSelector(examples=examples, example_prompt=example_prompt, max_length=25)

# Create a FewShotPromptTemplate
dynamic_prompt=FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt, 
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym",
    input_variables=["input"],
    example_separator="\n\n",
    )

# Generate a prompt using the format method
print(dynamic_prompt.format(input="big"))

# Advantages: Offers customization through various selectors and effective for managing a large number of examples
# Disadvantages: Involves manual creation and selection of examples


Give the antonym of every input

 Word: happy, Antonym: sad 

 Word: tall, Antonym: short 

 Word: energetic, Antonym: lethargic 

 Word: sunny, Antonym: gloomy 

Word: big
Antonym


SemanticSimilarityExampleSelector ->  Selects examples based on their semantic resemblance to the input. 

In [5]:
# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

In [6]:
# create Deep Lake dataset
my_activeloop_org_id = "srishtysuman2919"
my_activeloop_dataset_name = "langchain_Example_Seelctor"
dataset_path = "https:/app.activeloop.ai/srishtysuman2919/langchain_course_from_zero_to_hero"
db = DeepLake(dataset_path=dataset_path)

In [7]:
# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

Creating 5 embeddings in 1 batches of size 5:: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]

Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (5, 1)      str     None   
 metadata     json      (5, 1)      str     None   
 embedding  embedding  (5, 1536)  float32   None   
    id        text      (5, 1)      str     None   


In [9]:
# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:", 
    input_variables=["temperature"],
)


In [10]:
# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   
print(similar_prompt.format(temperature="30°C"))  

Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


In [11]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) 


# SemanticSimilarityExampleSelector-> 1. uses the Deep Lake vector store and OpenAIEmbeddings to measure semantic similarity. 
#                                     2. stores the samples on the database in the cloud, and retrieves similar samples.


Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:
